# First glance on Onetahi

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# Load the Excel file
file_path = '/home/leo/Documents/These/Données/Rearangement_OB2.xlsx'  # Replace with the path to your file
df = pd.read_excel(file_path)

## Cleanup and visualization

The focus is on the two aedes species present on the island

In [ ]:
# Filter rows where 'species' column is equal to 'ae_poly'
Poly_df = df[(df['Spc'] == 'ae_poly')]
#Aegipty_df = df[(df['Spc'] == 'ae_aeg') & (df['Motu'] == 'Onetahi')]

There's a first cleanup to do on the dates values. We'll convert them to datetime objects and check for missing values.

In [ ]:
Poly_df.loc[-82:, 'Date'].values

In [ ]:
Poly_df['Date'][-82:].values

In [ ]:
# Ensure 'Date' column is in datetime format
Poly_df['Date'][:-82] = pd.to_datetime(Poly_df['Date'][:-82])
#Aegipty_df.loc[:-80, 'Date'] = pd.to_datetime(Aegipty_df.loc[:-80, 'Date'])

# Convert month names to month numbers
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

date_poly = pd.DataFrame({
    'Day': Poly_df['Date'][-82:].values,
    'Month': Poly_df['Month'][-82:].map(month_map).values,
    'Year': [2024] * 82
})
#date_aeg = pd.DataFrame({
#    'Day': Aegipty_df['Date'][-80:].values,
#    'Month': Aegipty_df['Month'][-80:].map(month_map).values,
#    'Year': [2023] * 80
#})
#date_aeg[['Year', 'Month', 'Day']]

Poly_df['Date'][-82:] = pd.to_datetime(date_poly[['Year', 'Month', 'Day']])
#Aegipty_df['Date'][-80:] = pd.to_datetime(date_aeg[['Year', 'Month', 'Day']])

We now have to handle the Nan values. They are for now removed from the file (most of them are from Covid I think).

In [ ]:
# Split the dataset based on the 'action' column
df_poly_r = Poly_df[Poly_df['Action'] == 'Release'].copy()  # Rows where 'action' is 'Release'
df_poly_c = Poly_df[Poly_df['Action'] == 'Capture'].copy()  # Rows where 'action' is 'Capture'
#df_aegipty_c = Aegipty_df[Aegipty_df['Action'] == 'Capture'].copy()  # Rows where 'action' is 'Capture'

# Clean 'Nb_ind' column by replacing 'NA ' or NaN values with np.nan and dropping them
df_poly_r = df_poly_r.dropna(subset=['Nb_ind'])
df_poly_c['Nb_ind'] = df_poly_c['Nb_ind'].replace('NA ', np.nan)
df_poly_c = df_poly_c.dropna(subset=['Nb_ind'])
df_poly_c = df_poly_c.dropna(subset=['Date'])

#df_aegipty_c['Nb_ind'] = df_aegipty_c['Nb_ind'].replace('NA ', np.nan)
#df_aegipty_c = df_aegipty_c.dropna(subset=['Nb_ind'])
#df_aegipty_c = df_aegipty_c.dropna(subset=['Date'])

# Display the DataFrames
print(df_poly_r.head())
print(df_poly_c.head())
#print(df_aegipty_c.head())

In [ ]:
# Optionally, save the filtered data to a new csv file
#df_poly_r.to_csv('../Data/df_poly_release.csv', index=False)
#df_poly_c.to_csv('../Data/df_poly_capture.csv', index=False)
#df_aegipty_c.to_csv('../Data/df_aegipty_cature.csv', index=False)

More info on the number of captured and released mosquitoes

In [ ]:
# Get the numbre of mosquitoes released and captured 
nb_poly_mosquitoes_released = df_poly_r['Nb_ind'].sum()
nb_poly_mosquitoes_captured = df_poly_c['Nb_ind'].sum()
#nb_aegipty_mosquitoes_captured = df_aegipty_c['Nb_ind'].sum()

#print(f"Number of Ae. aegypti mosquitoes captured: {nb_aegipty_mosquitoes_captured}")
print(f"Number of Ae. polynesiensis mosquitoes captured: {nb_poly_mosquitoes_captured}")
print(f"Number of Ae. polynesiensis mosquitoes released: {nb_poly_mosquitoes_released}")

In [ ]:
# Add a new variable in the df_poly_* : phase ! The phase 0 is the dates before the release of the mosquitoes (2018-10-17). The second is between this date and 2019-11-13. The 3rd is between 2019-11-13 and 2021-08-03. The 4th is from 2022-09-23. The last from 2022-10-09 to the end 
df_poly_r['Phase'] = 'Capture'
df_poly_c['Phase'] = 'Capture'

# Convert string dates to datetime objects
release_dates = [
	pd.to_datetime('2018-10-16'),
	pd.to_datetime('2019-11-13'),
	pd.to_datetime('2021-08-04'),
	pd.to_datetime('2022-09-22'),
	pd.to_datetime('2022-10-08')
]

df_poly_r['Phase'] = np.where(df_poly_r['Date'] > release_dates[0], 'Capture + Release', 'Capture')
df_poly_c['Phase'] = np.where(df_poly_c['Date'] > release_dates[0], 'Capture + Release', 'Capture')

df_poly_r['Phase'] = np.where(df_poly_r['Date'] > release_dates[1], 'Capture', df_poly_r['Phase'])
df_poly_c['Phase'] = np.where(df_poly_c['Date'] > release_dates[1], 'Capture', df_poly_c['Phase'])

df_poly_r['Phase'] = np.where(df_poly_r['Date'] > release_dates[2], 'Covid', df_poly_r['Phase'])
df_poly_c['Phase'] = np.where(df_poly_c['Date'] > release_dates[2], 'Covid', df_poly_c['Phase'])

df_poly_r['Phase'] = np.where(df_poly_r['Date'] > release_dates[3], 'Capture', df_poly_r['Phase'])
df_poly_c['Phase'] = np.where(df_poly_c['Date'] > release_dates[3], 'Capture', df_poly_c['Phase'])

df_poly_r['Phase'] = np.where(df_poly_r['Date'] > release_dates[4], 'Capture + Release', df_poly_r['Phase'])
df_poly_c['Phase'] = np.where(df_poly_c['Date'] > release_dates[4], 'Capture + Release', df_poly_c['Phase'])

# Plot the number of mosquitoes captured by phase
fig, ax = plt.subplots()
sns.barplot(x='Phase', y='Nb_ind', data=df_poly_c, ax=ax)
ax.set_title('Number of Ae. polynesiensis mosquitoes captured by phase')
plt.show()

In [ ]:
df_poly_c['Event'] = 'Capture'
df_poly_c['Event'] = np.where(df_poly_c['Phase'] == '1', 'Capture + Release', df_poly_c['Event'])
df_poly_c['Event'] = np.where(df_poly_c['Phase'] == '4', 'Capture + Release', df_poly_c['Event'])


In [ ]:
# Do a matrix, with lines corresponding to Trap_Num and column to the Phase. The values in the matrix are the number of mosquitoes captured in each trap during each phase divided by the number of capture by this trap on the given phase.
# The matrix will be a 2D numpy array
trap_num = df_poly_c['Trap_num'].unique()
phase = df_poly_c['Phase'].unique()
matrix = np.zeros((len(trap_num), len(phase)))
for i, trap in enumerate(trap_num):
    for j, ph in enumerate(phase):
        matrix[i, j] = df_poly_c[(df_poly_c['Trap_num'] == trap) & (df_poly_c['Phase'] == ph)]['Nb_ind'].sum()

# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(matrix, annot=True, fmt='.0f', cmap='viridis', xticklabels=phase, yticklabels=trap_num)
plt.xlabel('Phase')
plt.ylabel('Trap number')
plt.title('Number of mosquitoes captured in each trap during each phase')
plt.show()


plots are great to have a better understanding. We can clearly see the impact of covid, and the change in the trap type after covid.

In [ ]:
# A little dataset which sums the captured mosqitoes by date, but doesn't sum the other variable and keeps the value of Event
tiny_df = df_poly_c.groupby(['Date', 'Phase', 'Event']).sum().reset_index()

In [ ]:
#list of the different motus
motus = df_poly_c['Motu'].unique()
motus

In [ ]:
# Get the max date in df_poly_c
max_date = df_poly_c['Date'].max()
max_date

In [ ]:
# Plot a double plot for the data for Capture events of poly depending on Phase. Use a different color for each phase
for i in range(len(motus)):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        data=df_poly_c[(df_poly_c['Motu'] == motus[i]) & (df_poly_c['Sex'] == 'F')], x='Date', y='Nb_ind', hue='Sex', s=50,
        palette={'F': 'green', 'M': 'red'}
    )
    plt.title(f'Number of Individuals by Date (Capture) - {motus[i]}')
    plt.xlabel('Date')
    plt.xlim(pd.to_datetime('2014-01-01'), pd.to_datetime('2025-01-01'))
    plt.ylabel('Number of Individuals')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    #plt.savefig('../img/capture_plot_poly.png')
    plt.show()


In [ ]:
#export tianaranu , honuea and oroatera and onetahi and rimatuu motus in a df, with the dates and the number of mosquitoes captured between 2019-10-02 00:00:00 and 2023-11-17 21:46:00. Put them all in a single df with the dates
tianaranu = df_poly_c[df_poly_c['Motu'] == 'Tianaranu']
honuea = df_poly_c[df_poly_c['Motu'] == 'Honuea']
oroatera = df_poly_c[df_poly_c['Motu'] == 'Oroatera']
onetahi = df_poly_c[df_poly_c['Motu'] == 'Onetahi']
rimatuu = df_poly_c[df_poly_c['Motu'] == 'Rimatuu']

# Combine all dataframes into a single dataframe
combined_df = pd.concat([tianaranu, honuea, oroatera, onetahi, rimatuu], axis=0)
# Keep only Date (between 2019-10-02 00:00:00 and 2023-11-17 21:46:00.), motu, Nb_ind when Sex == Female
combined_df = combined_df[(combined_df['Date'] >= pd.to_datetime('2019-10-02'))]
combined_df = combined_df[combined_df['Sex'] == 'F']
combined_df = combined_df[['Date', 'Motu', 'Nb_ind']]

# Reset index
combined_df.reset_index(drop=True, inplace=True)
combined_df.to_csv('../Data/combined_poly_capture.csv', index=False)

In [ ]:
combined_df.head()

# Meteo

In [ ]:
#open the meteo data, the values are separated by ;

meteo_path = '/home/leo/Documents/These/Données/Meteo/Q_987_previous-1950-2023_RR-T-Vent.csv'
meteo_df = pd.read_csv(meteo_path, sep=';')
# Print the column names to check if they match
print(meteo_df.columns)

#get the other dataset with the meteo data in the same format and put it in meteo_df
meteo_path2 = '/home/leo/Documents/These/Données/Meteo/Q_987_previous-1950-2023_autres-parametres.csv'
meteo_df2 = pd.read_csv(meteo_path2, sep=';')
# Print the column names to check if they match
print(meteo_df2.columns)

# keep only the values where 'NOM_USUEL' == 'TETIAROA 1'
meteo_df = meteo_df[meteo_df['NOM_USUEL'] == 'TETIAROA 1']
meteo_df2 = meteo_df2[meteo_df2['NOM_USUEL'] == 'TETIAROA 1']


meteo_df2.head()

In [ ]:
# Keep only the AAAAMMJJ, RR, TX, TM and FXI2 columns
meteo_df = meteo_df[['AAAAMMJJ', 'RR', 'TX', 'TM', 'FFM', 'TN']]

# Keep AAAAMMJJ, GLOT, UM UN UX in df2
meteo_df2 = meteo_df2[['AAAAMMJJ', 'GLOT', 'UM', 'UN', 'UX']]

# Convert the 'AAAAMMJJ' column to datetime format and remove all values where the date is before 2014-01-01, same for df2
meteo_df['AAAAMMJJ'] = pd.to_datetime(meteo_df['AAAAMMJJ'], format='%Y%m%d')
meteo_df = meteo_df[meteo_df['AAAAMMJJ'] >= pd.to_datetime('2014-01-01')]

meteo_df2['AAAAMMJJ'] = pd.to_datetime(meteo_df2['AAAAMMJJ'], format='%Y%m%d')
meteo_df2 = meteo_df2[meteo_df2['AAAAMMJJ'] >= pd.to_datetime('2014-01-01')]

In [ ]:
#merge the two
meteo_df = pd.merge(meteo_df, meteo_df2, on='AAAAMMJJ', how='inner')
meteo_df.head()

# Replace missing values wwith np.nan
meteo_df = meteo_df.replace('mq', np.nan)

In [ ]:
# Take the average of each column for each day
meteo_df_mean = meteo_df.groupby('AAAAMMJJ').mean().reset_index()

# Now take the min and max values for each day
meteo_df_min = meteo_df.groupby('AAAAMMJJ').min().reset_index()
meteo_df_max = meteo_df.groupby('AAAAMMJJ').max().reset_index()

# Now do a single data set with the sum of RR for each day, only with the RR column
meteo_df_sum = meteo_df.groupby('AAAAMMJJ')['RR'].sum().reset_index()
meteo_df_sum['precip_1week'] = meteo_df_sum['RR'].rolling(window=7, min_periods=1).sum()
meteo_df_sum['precip_2weeks'] = meteo_df_sum['RR'].rolling(window=14, min_periods=1).sum()

# Take a sliding mean that take sinto account the previous two days and the next wo days for TM
meteo_df_min['TM_sliding_mean'] = meteo_df_min['TM'].rolling(window=5, min_periods=1).mean()
meteo_df_mean['TM_sliding_mean'] = meteo_df_mean['TM'].rolling(window=5, min_periods=1).mean()

# Same for GLOT but taking the previous 6 days
meteo_df_min['GLOT_sliding_mean'] = meteo_df_min['GLOT'].rolling(window=7, min_periods=1, center=False).mean()
meteo_df_mean['GLOT_sliding_mean'] = meteo_df_mean['GLOT'].rolling(window=7, min_periods=1, center=False).mean()

In [ ]:
#get the date for the max of captured mosquitoes in 2021 without using dt.year
max_date = df_poly_c[(pd.to_datetime('2021-01-01') < df_poly_c['Date']) & (df_poly_c['Date'] < pd.to_datetime('2022-01-01'))].groupby('Date')['Nb_ind'].sum().idxmax()
print(max_date)

In [ ]:
fig, axs = plt.subplots(2, 5, figsize=(20, 10))
max_date = df_poly_c[(pd.to_datetime('2015-01-01') < df_poly_c['Date']) & (df_poly_c['Date'] < pd.to_datetime('2022-01-01'))].groupby('Date')['Nb_ind'].sum().idxmax()

date_min = pd.to_datetime('2019-01-01')
date_max = pd.to_datetime('2022-02-01')
# Plot 1: Number of Individuals by Date (Capture)
sns.scatterplot(data=df_poly_c, x='Date', y='Nb_ind', s=20, ax=axs[0, 0])
axs[0, 0].set_title('Number of Individuals by Date (Capture)')
axs[0, 0].set_xlabel('Date')
axs[0, 0].set_xlim(date_min, date_max)
axs[0, 0].set_ylabel('Number of Individuals')
axs[0, 0].grid(True)
axs[0, 0].tick_params(axis='x', rotation=45)
axs[0, 0].axvline(max_date, color='red', linestyle='--')

# Plot 2: Temperature minimale par jour
sns.scatterplot(data=meteo_df_min, x='AAAAMMJJ', y='TM', s=20, ax=axs[0, 1])
axs[0, 1].set_title('Temperature minimale par jour')
axs[0, 1].set_xlabel('Date')
axs[0, 1].set_xlim(date_min, date_max)
axs[0, 1].set_ylabel('Temperature')
axs[0, 1].grid(True)
axs[0, 1].tick_params(axis='x', rotation=45)
axs[0, 1].axvline(max_date, color='red', linestyle='--')

# Plot 3: Température Moyenne par jour
sns.scatterplot(data=meteo_df_mean, x='AAAAMMJJ', y='TM', s=20, ax=axs[0, 2])
axs[0, 2].set_title('Température Moyenne par jour')
axs[0, 2].set_xlabel('Date')
axs[0, 2].set_xlim(date_min, date_max)
axs[0, 2].set_ylabel('Temperature moyenne')
axs[0, 2].grid(True)
axs[0, 2].tick_params(axis='x', rotation=45)
axs[0, 2].axvline(max_date, color='red', linestyle='--')

# Plot 4: Précipitation par jour
sns.scatterplot(data=meteo_df_sum, x='AAAAMMJJ', y='RR', s=20, ax=axs[0, 3])
axs[0, 3].set_title('Précipitation par jour')
axs[0, 3].set_xlabel('Date')
axs[0, 3].set_xlim(date_min, date_max)
axs[0, 3].set_ylabel('Précipitation')
axs[0, 3].grid(True)
axs[0, 3].tick_params(axis='x', rotation=45)
axs[0, 3].axvline(max_date, color='red', linestyle='--')

# Plot 5: FFM values
sns.scatterplot(data=meteo_df_mean, x='AAAAMMJJ', y='TM_sliding_mean', s=20, ax=axs[0, 4])
axs[0, 4].set_title('Temperature moyenne avec moyenne glissante par jour')
axs[0, 4].set_xlabel('Date')
axs[0, 4].set_xlim(date_min, date_max)
axs[0, 4].set_ylabel('FFM')
axs[0, 4].grid(True)
axs[0, 4].tick_params(axis='x', rotation=45)
axs[0, 4].axvline(max_date, color='red', linestyle='--')

# Plot 6: Temperature mpyenne glissante
sns.scatterplot(data=meteo_df_min, x='AAAAMMJJ', y='TM_sliding_mean', s=20, ax=axs[1, 0])
axs[1, 0].set_title('Temperature minimale avec moyenne glissante par jour')
axs[1, 0].set_xlabel('Date')
axs[1, 0].set_xlim(date_min, date_max)
axs[1, 0].set_ylabel('Temperature')
axs[1, 0].grid(True)
axs[1, 0].tick_params(axis='x', rotation=45)
axs[1, 0].axvline(max_date, color='red', linestyle='--')

# Plot 7: FFM values
sns.scatterplot(data=meteo_df_mean, x='AAAAMMJJ', y='UM', s=20, ax=axs[1, 1])
axs[1, 1].set_title('Humidité moyenne par jour')
axs[1, 1].set_xlabel('Date')
axs[1, 1].set_xlim(date_min, date_max)
axs[1, 1].set_ylabel('Humidité (%)')
axs[1, 1].grid(True)
axs[1, 1].tick_params(axis='x', rotation=45)
axs[1, 1].axvline(max_date, color='red', linestyle='--')

# Plot 8: Temperature mpyenne glissante
sns.scatterplot(data=meteo_df_min, x='AAAAMMJJ', y='UN', s=20, ax=axs[1, 2])
axs[1, 2].set_title('Humidité minimale par jour')
axs[1, 2].set_xlabel('Date')
axs[1, 2].set_xlim(date_min, date_max)
axs[1, 2].set_ylabel('Humidité (%)')
axs[1, 2].grid(True)
axs[1, 2].tick_params(axis='x', rotation=45)
axs[1, 2].axvline(max_date, color='red', linestyle='--')

# Plot 9: FFM values
sns.scatterplot(data=meteo_df_sum, x='AAAAMMJJ', y='precip_1week', s=20, ax=axs[1, 3])
axs[1, 3].set_title('Cumul de précipitations sur 1 semaine')
axs[1, 3].set_xlabel('Date')
axs[1, 3].set_xlim(date_min, date_max)
axs[1, 3].set_ylabel('Humidité (%)')
axs[1, 3].grid(True)
axs[1, 3].tick_params(axis='x', rotation=45)
axs[1, 3].axvline(max_date, color='red', linestyle='--')

# Plot 10: Temperature mpyenne glissante
sns.scatterplot(data=meteo_df_sum, x='AAAAMMJJ', y='precip_2weeks', s=20, ax=axs[1, 4])
axs[1, 4].set_title('Cumul de précipitations sur 2 semaines')
axs[1, 4].set_xlabel('Date')
axs[1, 4].set_xlim(date_min, date_max)
axs[1, 4].set_ylabel('Ensoleillement (J/cm2)')
axs[1, 4].grid(True)
axs[1, 4].tick_params(axis='x', rotation=45)
axs[1, 4].axvline(max_date, color='red', linestyle='--')

# save the plot to a file
plt.savefig('../img/meteo_plot.png')
plt.tight_layout()
plt.show()


# Study of the RR variable

In [ ]:
meteo_df = meteo_df[['AAAAMMJJ', 'RR']]
meteo_df.head()

meteo_df_sum = meteo_df_sum[['AAAAMMJJ', 'precip_1week', 'precip_2weeks']]
meteo_df_sum.head()

In [ ]:
#plot the precip_2weeks and evolution of capured individuals on tetiaroa
date_min = pd.to_datetime('2019-05-02')
date_max = pd.to_datetime('2021-12-31')
fig, axs = plt.subplots(2, 1, figsize=(10, 10))
# Plot 1: Number of Individuals by Date (Capture)
sns.scatterplot(data=tiny_df, x='Date', y='Nb_ind', s=20, ax=axs[0])
axs[0].set_title('Number of Individuals by Date (Capture) on tetiaroa')
axs[0].set_xlabel('Date')
axs[0].set_xlim(date_min, date_max)
axs[0].set_ylabel('Number of Individuals')
axs[0].grid(True)
axs[0].tick_params(axis='x', rotation=45)
axs[0].axvline(max_date, color='red', linestyle='--')
# Plot 2: Precipitation by Date
sns.scatterplot(data=meteo_df_sum, x='AAAAMMJJ', y='precip_2weeks', s=20, ax=axs[1])
axs[1].set_title('Cumul de précipitations sur 2 semaines')
axs[1].set_xlabel('Date')
axs[1].set_xlim(date_min, date_max)
axs[1].set_ylabel('Cumul de précipitations (mm)')
axs[1].grid(True)
axs[1].tick_params(axis='x', rotation=45)
axs[1].axvline(max_date, color='red', linestyle='--')
plt.tight_layout()
#plt.savefig('../img/meteo_plot2.png')
plt.show()

In [ ]:
# same on honuea
fig, axs = plt.subplots(2, 1, figsize=(10, 10))
# Plot 1: Number of Individuals by Date (Capture)
sns.scatterplot(data=tiny_df_honuea, x='Date', y='Nb_ind', s=20, ax=axs[0])
axs[0].set_title('Number of Individuals by Date (Capture) on honuea')
axs[0].set_xlabel('Date')
axs[0].set_xlim(date_min, date_max)
axs[0].set_ylabel('Number of Individuals')
axs[0].grid(True)
axs[0].tick_params(axis='x', rotation=45)
axs[0].axvline(max_date, color='red', linestyle='--')
# Plot 2: Precipitation by Date
sns.scatterplot(data=meteo_df_sum, x='AAAAMMJJ', y='precip_2weeks', s=20, ax=axs[1])
axs[1].set_title('Cumul de précipitations sur 2 semaines')
axs[1].set_xlabel('Date')
axs[1].set_xlim(date_min, date_max)
axs[1].set_ylabel('Cumul de précipitations (mm)')
axs[1].grid(True)
axs[1].tick_params(axis='x', rotation=45)
axs[1].axvline(max_date, color='red', linestyle='--')
plt.tight_layout()
#plt.savefig('../img/meteo_plot_honuea.png')
plt.show()